In [1]:
import pandas as pd
import mysql.connector
from tabulate import tabulate 

In [ ]:
df=pd.read_csv("C:\\Users\\sidst\\Downloads\\vehicle_check.csv")
print(df.isnull().sum()) #null values found in search_type


In [13]:
#changing null values to meaningful values
df.loc[(df['search_conducted']==False)&(df['search_type'].isna()),'search_type']="No search"
df['search_type'].mode() #frisk
df.loc[(df['search_conducted']==True)&(df['search_type'].isna()),'search_type']="Unknown"

#checking anymore null values left to handle in search_type col, res=0
df['search_type'].isna().sum()

#checking whole table for null values, res=0
df.isna().sum()

#checking vehicle number
df['vehicle_number'].is_unique

#merging two columns for easy analysis and dropping unneccassary cols
df['stop_date']=pd.to_datetime(df['stop_date'], format="%Y-%m-%d")
df['stop_datetime']=pd.to_datetime(df['stop_date'].astype(str)+" "+df['stop_time'])
df.drop('stop_time', axis=1, inplace=True)
df.drop('stop_date', axis=1, inplace=True)



In [14]:
def dur_in_min(dur_str):
  if "-" in dur_str:
    s,e=map(int,dur_str.split("-"))
    return e-s
  elif "+" in dur_str:
    return int(dur_str.replace('+',''))
  else:
    try:
      return int(dur_str)
    except:
      return 0

In [15]:
#changing col values suitable for analysis
df['stop_duration'].str.endswith('Min').sum()
df['stop_duration']=df['stop_duration'].str.replace('Min','')

df['stop_duration_min']=df['stop_duration'].apply(dur_in_min)
df.drop('stop_duration', axis=1, inplace=True)


In [17]:
#writinh cleaned data into csv file
df.to_csv("vehicle_check_cleaned.csv", index=False)
df1=pd.read_csv("vehicle_check_cleaned.csv")

df1.dtypes
#changing datatypes
df1['stop_datetime']=pd.to_datetime(df1['stop_datetime'], format="%Y-%m-%d %H:%M:%S")
df1.dtypes


country_name                  object
driver_gender                 object
driver_age_raw                 int64
driver_age                     int64
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
search_type                   object
stop_outcome                  object
is_arrested                     bool
drugs_related_stop              bool
vehicle_number                object
stop_datetime         datetime64[ns]
stop_duration_min              int64
dtype: object

In [ ]:
datatype_mapping={
    'object':'text',
    'int64':'int',
    'bool':'boolean',
    'datetime64[ns]':'datetime'
}
column_list=[]
for col in df1.columns:
   typ=str(df1[col].dtype)
   column_list.append(col+" "+datatype_mapping[typ])

col_syntax=",".join(column_list)
tab_name="police_log_tab"
create_query=f"CREATE TABLE IF NOT EXISTS {tab_name} ({col_syntax})"
print(create_query)


try:
  connection=mysql.connector.connect(
      host = "gateway01.eu-central-1.prod.aws.tidbcloud.com",
      port = 4000,
      user = "4HRQRPypJvdSYNG.root",
      password = "IOTAQOIa0EjHZU5V",
      database="guvi"
    )
  mycursor=connection.cursor(buffered=True)
  print("Connected to database")

  mycursor.execute(create_query)
  print("Table created")

  ph=["%s"]*len(df1.columns)
  place_holder=",".join(ph)
  insert_query=f"INSERT INTO {tab_name} VALUES({place_holder})"
  all_rows = []
  for val in df1.values:
     row = []
     for i in val:
        if hasattr(i, "item"):
            row.append(i.item())
        elif isinstance(i, pd.Timestamp):
            row.append(i.to_pydatetime())
        else:
            row.append(i)
     all_rows.append(tuple(row))

# Insert in batch
  mycursor.executemany(insert_query, all_rows)
  connection.commit()

except Exception as e:
  print("Error connecting database"+str(e))
mycursor.close()
connection.close()


CREATE TABLE IF NOT EXISTS police_log (country_name text,driver_gender text,driver_age_raw int,driver_age int,driver_race text,violation_raw text,violation text,search_conducted boolean,search_type text,stop_outcome text,is_arrested boolean,drugs_related_stop boolean,vehicle_number text,stop_datetime datetime,stop_duration_min int)


In [2]:
import mysql.connector
from tabulate import tabulate

connection=mysql.connector.connect(
      host = "gateway01.eu-central-1.prod.aws.tidbcloud.com",
      port = 4000,
      user = "4HRQRPypJvdSYNG.root",
      password = "IOTAQOIa0EjHZU5V",
      database="guvi"
    )
mycursor=connection.cursor(buffered=True)

In [20]:
mycursor.execute("SHOW INDEXES FROM police_log")
for row in mycursor.fetchall():
    print(row)

('police_log', 1, 'idx_country_violation', 1, 'country_name', 'A', 3, 50, None, 'YES', 'BTREE', '', '', 'YES', None, 'NO')
('police_log', 1, 'idx_country_violation', 2, 'violation', 'A', 5, 50, None, 'YES', 'BTREE', '', '', 'YES', None, 'NO')
('police_log', 1, 'idx_drug_stop', 1, 'drugs_related_stop', 'A', 2, None, None, 'YES', 'BTREE', '', '', 'YES', None, 'NO')
('police_log', 1, 'idx_search_conducted', 1, 'search_conducted', 'A', 2, None, None, 'YES', 'BTREE', '', '', 'YES', None, 'NO')
('police_log', 1, 'idx_is_arrested', 1, 'is_arrested', 'A', 2, None, None, 'YES', 'BTREE', '', '', 'YES', None, 'NO')
('police_log', 1, 'idx_stop_datetime', 1, 'stop_datetime', 'A', 65400, None, None, 'YES', 'BTREE', '', '', 'YES', None, 'NO')


In [21]:
#1.What are the top 10 vehicle_Number involved in drug-related stops

mycursor.execute("SELECT vehicle_number, COUNT(vehicle_number) AS vehicle_count  FROM police_log where drugs_related_stop=1 GROUP BY vehicle_number ORDER BY vehicle_count DESC LIMIT 10")


all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+------------------+-----------------+
| vehicle_number   |   vehicle_count |
|------------------+-----------------|
| RJ51LW6272       |               1 |
| TN82FI4218       |               1 |
| KA75LG8145       |               1 |
| WB67UM9183       |               1 |
| MH28GS1779       |               1 |
| KA21NT9797       |               1 |
| DL28OO9477       |               1 |
| MH59BZ5348       |               1 |
| TN60ND8984       |               1 |
| DL10UB5265       |               1 |
+------------------+-----------------+


In [22]:
#2.Which vehicles were most frequently searched?

mycursor.execute("SELECT vehicle_number, COUNT(vehicle_number) AS vehicle_count FROM police_log WHERE search_conducted=1 GROUP BY vehicle_number ORDER BY vehicle_count DESC")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))


+------------------+-----------------+
| vehicle_number   |   vehicle_count |
|------------------+-----------------|
| KA13OP1777       |               1 |
| WB68RM6613       |               1 |
| GJ18TU3963       |               1 |
| UP63XW9413       |               1 |
| KA36TZ1949       |               1 |
| GJ42GI9536       |               1 |
| MH68BJ2422       |               1 |
| WB48EI5497       |               1 |
| UP51MF4587       |               1 |
| RJ39WZ6977       |               1 |
| DL97AQ5436       |               1 |
| GJ53YY5132       |               1 |
| KA92OI8242       |               1 |
| KA48ZN3913       |               1 |
| KA54KF1031       |               1 |
| UP87CU5242       |               1 |
| UP89GO1654       |               1 |
| GJ89GM9223       |               1 |
| GJ60XK5372       |               1 |
| MH69KV8149       |               1 |
| GJ25DB5507       |               1 |
| KA63WH3644       |               1 |
| WB17CK4141       |     

In [23]:
#4.Which driver age group had the highest arrest rate?

mycursor.execute("SELECT driver_age, (COUNT(CASE WHEN is_arrested=1 THEN 1 END) / COUNT(driver_age))* 100.0 AS arrest_rate FROM police_log GROUP BY driver_age ORDER BY arrest_rate DESC LIMIT 1")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------------+---------------+
|   driver_age |   arrest_rate |
|--------------+---------------|
|           53 |       53.4634 |
+--------------+---------------+


In [24]:
#5.What is the gender distribution of drivers stopped in each country?

mycursor.execute("SELECT country_name, driver_gender, COUNT(*) AS stop_count FROM police_log GROUP BY country_name, driver_gender ORDER BY country_name")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+-----------------+--------------+
| country_name   | driver_gender   |   stop_count |
|----------------+-----------------+--------------|
| Canada         | F               |        10997 |
| Canada         | M               |        10911 |
| India          | M               |        10955 |
| India          | F               |        11043 |
| USA            | F               |        10841 |
| USA            | M               |        10791 |
+----------------+-----------------+--------------+


In [25]:
#6.Which race and gender combination has the highest search rate?

mycursor.execute("SELECT driver_race, driver_gender, (COUNT(CASE WHEN search_conducted=1 THEN 1 END) / COUNT(*))*100.0 AS search_rate FROM police_log GROUP BY driver_race, driver_gender ORDER BY search_rate DESC LIMIT 1")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+---------------+-----------------+---------------+
| driver_race   | driver_gender   |   search_rate |
|---------------+-----------------+---------------|
| Asian         | F               |       51.2859 |
+---------------+-----------------+---------------+


In [26]:
#7.What time of day sees the most traffic stops?

mycursor.execute("""SELECT CASE WHEN HOUR(stop_datetime) BETWEEN 5 AND 11 THEN 'Morning'
                                WHEN HOUR(stop_datetime) BETWEEN 12 AND 16 THEN 'Afternoon'
                                WHEN HOUR(stop_datetime) BETWEEN 17 AND 20 THEN 'Evening'
                                ELSE 'Night'
                            END AS time_of_day, COUNT(*) AS stop_count
                    FROM police_log GROUP BY time_of_day ORDER BY stop_count DESC""")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+---------------+--------------+
| time_of_day   |   stop_count |
|---------------+--------------|
| Night         |        21900 |
| Morning       |        19320 |
| Afternoon     |        13518 |
| Evening       |        10800 |
+---------------+--------------+


In [27]:
#8.What is the average stop duration for different violations?

mycursor.execute("SELECT violation, AVG(stop_duration_min) AS avg_stop_duration FROM police_log GROUP BY violation ORDER BY avg_stop_duration DESC")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+-------------+---------------------+
| violation   |   avg_stop_duration |
|-------------+---------------------|
| Other       |             19.9416 |
| Seatbelt    |             19.6787 |
| DUI         |             19.6363 |
| Signal      |             19.6097 |
| Speeding    |             19.6011 |
+-------------+---------------------+


In [28]:
#9. Are stops during the night more likely to lead to arrests?

mycursor.execute("""SELECT CASE WHEN HOUR(stop_datetime) BETWEEN 5 AND 11 THEN 'Morning'
                                WHEN HOUR(stop_datetime) BETWEEN 12 AND 16 THEN 'Afternoon'
                                WHEN HOUR(stop_datetime) BETWEEN 17 AND 20 THEN 'Evening'
                                ELSE 'Night'
                            END AS time_of_day, (COUNT(CASE WHEN is_arrested=1 THEN 1 END)/ COUNT(*))* 100.0 AS arrest_rate
                    FROM police_log GROUP BY time_of_day ORDER BY arrest_rate DESC""")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))


+---------------+---------------+
| time_of_day   |   arrest_rate |
|---------------+---------------|
| Morning       |       50.7971 |
| Evening       |       50.3796 |
| Afternoon     |       49.8742 |
| Night         |       49.5388 |
+---------------+---------------+


In [29]:
#10.Which violations are most associated with searches or arrests?

mycursor.execute("SELECT violation, COUNT(*) AS no_of_count FROM police_log WHERE is_arrested=1 or search_conducted=1 GROUP BY violation ORDER BY no_of_count DESC")

mycursor.execute("""SELECT violation,
                           COUNT(CASE WHEN is_arrested=1 THEN 1 END) AS arrest_count,
                           COUNT(CASE WHEN search_conducted=1 THEN 1 END) AS search_count
                    FROM police_log GROUP BY violation ORDER BY (search_count+arrest_count) DESC""")


all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))


+-------------+----------------+----------------+
| violation   |   arrest_count |   search_count |
|-------------+----------------+----------------|
| Speeding    |           6598 |           6573 |
| Seatbelt    |           6573 |           6562 |
| Other       |           6516 |           6576 |
| DUI         |           6610 |           6481 |
| Signal      |           6549 |           6509 |
+-------------+----------------+----------------+


In [30]:
#11.Which violations are most common among younger drivers (<25)?

mycursor.execute("SELECT violation, COUNT(*) AS count FROM police_log WHERE driver_age < 25 GROUP BY violation ORDER BY count DESC")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+-------------+---------+
| violation   |   count |
|-------------+---------|
| Speeding    |    1476 |
| Signal      |    1427 |
| Other       |    1422 |
| Seatbelt    |    1420 |
| DUI         |    1359 |
+-------------+---------+


In [31]:
#12. Is there a violation that rarely results in search or arrest?

mycursor.execute("""SELECT violation,
                           COUNT(*) AS stop_count,
                           (COUNT(CASE WHEN is_arrested=1 THEN 1 END)/ COUNT(*))*100.0 AS arrest_rate,
                           (COUNT(CASE WHEN search_conducted=1 THEN 1 END)/ COUNT(*))*100.0 AS search_rate
                    FROM police_log GROUP BY violation ORDER BY (arrest_rate+search_rate) ASC""")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+-------------+--------------+---------------+---------------+
| violation   |   stop_count |   arrest_rate |   search_rate |
|-------------+--------------+---------------+---------------|
| Other       |        13194 |       49.3861 |       49.8408 |
| Signal      |        13112 |       49.9466 |       49.6416 |
| DUI         |        13075 |       50.5545 |       49.5679 |
| Speeding    |        13150 |       50.1749 |       49.9848 |
| Seatbelt    |        13007 |       50.5343 |       50.4498 |
+-------------+--------------+---------------+---------------+


In [32]:
#13.Which countries report the highest rate of drug-related stops?

mycursor.execute("SELECT country_name, COUNT(*) AS tot_stops, COUNT(CASE WHEN drugs_related_stop=1 THEN 1 END) AS drug_stops, (COUNT(CASE WHEN drugs_related_stop=1 THEN 1 END)/COUNT(*))*100.0 AS highest_rate FROM police_log GROUP BY country_name ORDER BY highest_rate DESC")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+-------------+--------------+----------------+
| country_name   |   tot_stops |   drug_stops |   highest_rate |
|----------------+-------------+--------------+----------------|
| USA            |       21632 |        10896 |        50.3698 |
| Canada         |       21908 |        10976 |        50.1004 |
| India          |       21998 |        10897 |        49.5363 |
+----------------+-------------+--------------+----------------+


In [33]:
#14. What is the arrest rate by country and violation?

mycursor.execute("SELECT country_name, violation, COUNT(*) AS tot_stops, COUNT(CASE WHEN is_arrested=1 THEN 1 END) AS arrest_count, (COUNT(CASE WHEN is_arrested=1 THEN 1 END)/COUNT(*))*100.0 AS arrest_rate FROM police_log GROUP BY country_name, violation ORDER BY arrest_rate DESC")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+-------------+-------------+----------------+---------------+
| country_name   | violation   |   tot_stops |   arrest_count |   arrest_rate |
|----------------+-------------+-------------+----------------+---------------|
| Canada         | DUI         |        4414 |           2257 |       51.1328 |
| India          | DUI         |        4420 |           2247 |       50.8371 |
| India          | Speeding    |        4358 |           2212 |       50.7572 |
| India          | Signal      |        4444 |           2251 |       50.6526 |
| USA            | Seatbelt    |        4241 |           2145 |       50.5777 |
| India          | Seatbelt    |        4345 |           2197 |       50.5639 |
| Canada         | Seatbelt    |        4421 |           2231 |       50.4637 |
| Canada         | Other       |        4371 |           2198 |       50.286  |
| USA            | Signal      |        4341 |           2170 |       49.9885 |
| Canada         | Speeding    |        

In [34]:
#15.Which country has the most stops with search conducted?

mycursor.execute("SELECT country_name, COUNT(*) AS search_count FROM police_log WHERE search_conducted=1 GROUP BY country_name ORDER BY search_count DESC")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+----------------+
| country_name   |   search_count |
|----------------+----------------|
| Canada         |          11020 |
| India          |          10959 |
| USA            |          10722 |
+----------------+----------------+


In [ ]:
#1.Yearly Breakdown of Stops and Arrests by Country (Using Subquery and Window Functions)

mycursor.execute("""
SELECT
    country_name,
    year,
    tot_stops,
    arrest_count,
    (arrest_count / tot_stops) * 100 AS arrest_rate,
    SUM(arrest_count) OVER(PARTITION BY country_name ORDER BY year) AS arrest_tot
FROM (
    SELECT
        country_name,
        YEAR(stop_datetime) AS year,
        COUNT(CASE WHEN is_arrested = 1 THEN 1 END) AS arrest_count,
        COUNT(*) AS tot_stops
    FROM police_log
    GROUP BY country_name, YEAR(stop_datetime)
) AS year_table
ORDER BY country_name, year
""")


all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+--------+-------------+----------------+---------------+--------------+
| country_name   |   year |   tot_stops |   arrest_count |   arrest_rate |   arrest_tot |
|----------------+--------+-------------+----------------+---------------+--------------|
| Canada         |   2020 |       21908 |          10999 |       50.2054 |        10999 |
| India          |   2020 |       21998 |          11091 |       50.4182 |        11091 |
| USA            |   2020 |       21632 |          10756 |       49.7226 |        10756 |
+----------------+--------+-------------+----------------+---------------+--------------+


In [43]:
#2. Driver Violation Trends Based on Age and Race (Join with Subquery)

mycursor.execute("""with age_table as
(SELECT DISTINCT driver_race,driver_age
FROM police_log )
SELECT age_table.driver_race, age_table.driver_age, p.violation, COUNT(p.violation) AS violation_count
FROM age_table
JOIN police_log AS p
ON age_table.driver_race = p.driver_race AND age_table.driver_age = p.driver_age
GROUP BY age_table.driver_race, age_table.driver_age, p.violation
ORDER BY age_table.driver_race, age_table.driver_age, violation_count DESC
 """)



all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+---------------+--------------+-------------+-------------------+
| driver_race   |   driver_age | violation   |   violation_count |
|---------------+--------------+-------------+-------------------|
| Asian         |           18 | Speeding    |                50 |
| Asian         |           18 | Signal      |                45 |
| Asian         |           18 | Other       |                35 |
| Asian         |           18 | DUI         |                33 |
| Asian         |           18 | Seatbelt    |                29 |
| Asian         |           19 | DUI         |                47 |
| Asian         |           19 | Seatbelt    |                44 |
| Asian         |           19 | Speeding    |                39 |
| Asian         |           19 | Other       |                37 |
| Asian         |           19 | Signal      |                29 |
| Asian         |           20 | Signal      |                37 |
| Asian         |           20 | Speeding    |                

In [37]:
#3.Time Period Analysis of Stops (Joining with Date Functions) , Number of Stops by Year,Month, Hour of the Day

mycursor.execute("""
WITH time_data AS (
    SELECT
        YEAR(stop_datetime) AS year,
        MONTH(stop_datetime) AS month,
        HOUR(stop_datetime) AS hour
    FROM police_log
)
SELECT
    year,
    month,
    hour,
    COUNT(*) AS total_stops
FROM time_data
GROUP BY year, month, hour
ORDER BY year, month, hour;
""")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------+---------+--------+---------------+
|   year |   month |   hour |   total_stops |
|--------+---------+--------+---------------|
|   2020 |       1 |      0 |          1860 |
|   2020 |       1 |      1 |          1860 |
|   2020 |       1 |      2 |          1860 |
|   2020 |       1 |      3 |          1860 |
|   2020 |       1 |      4 |          1860 |
|   2020 |       1 |      5 |          1860 |
|   2020 |       1 |      6 |          1860 |
|   2020 |       1 |      7 |          1860 |
|   2020 |       1 |      8 |          1860 |
|   2020 |       1 |      9 |          1860 |
|   2020 |       1 |     10 |          1860 |
|   2020 |       1 |     11 |          1860 |
|   2020 |       1 |     12 |          1860 |
|   2020 |       1 |     13 |          1860 |
|   2020 |       1 |     14 |          1860 |
|   2020 |       1 |     15 |          1860 |
|   2020 |       1 |     16 |          1860 |
|   2020 |       1 |     17 |          1860 |
|   2020 |       1 |     18 |     

In [38]:
#4.Violations with High Search and Arrest Rates (Window Function)

mycursor.execute("""WITH rate_table as
                    (SELECT violation,
                           (COUNT(CASE WHEN search_conducted=1 THEN 1 END) / COUNT(*)) * 100.0 AS search_rate,
                           (COUNT(CASE WHEN is_arrested=1 THEN 1 END) / COUNT(*)) * 100.0 AS arrest_rate
                    FROM police_log
                    GROUP BY violation)
                    SELECT violation, search_rate, arrest_rate, RANK() OVER(ORDER BY (search_rate + arrest_rate) DESC) AS rate_rank
                    FROM rate_table
                    """)


all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+-------------+---------------+---------------+-------------+
| violation   |   search_rate |   arrest_rate |   rate_rank |
|-------------+---------------+---------------+-------------|
| Seatbelt    |       50.4498 |       50.5343 |           1 |
| Speeding    |       49.9848 |       50.1749 |           2 |
| DUI         |       49.5679 |       50.5545 |           3 |
| Signal      |       49.6416 |       49.9466 |           4 |
| Other       |       49.8408 |       49.3861 |           5 |
+-------------+---------------+---------------+-------------+


In [39]:
#5.Driver Demographics by Country (Age, Gender, and Race)

mycursor.execute("""SELECT country_name, driver_age, driver_gender, driver_race, COUNT(*) AS count, (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY country_name)) AS pct_of_country
                    FROM police_log GROUP BY country_name, driver_race, driver_age, driver_gender ORDER BY country_name, driver_race, driver_age DESC""")

all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+--------------+-----------------+---------------+---------+------------------+
| country_name   |   driver_age | driver_gender   | driver_race   |   count |   pct_of_country |
|----------------+--------------+-----------------+---------------+---------+------------------|
| Canada         |           80 | F               | Asian         |      42 |          0.19171 |
| Canada         |           80 | M               | Asian         |      38 |          0.17345 |
| Canada         |           79 | M               | Asian         |      35 |          0.15976 |
| Canada         |           79 | F               | Asian         |      42 |          0.19171 |
| Canada         |           78 | M               | Asian         |      35 |          0.15976 |
| Canada         |           78 | F               | Asian         |      32 |          0.14607 |
| Canada         |           77 | F               | Asian         |      33 |          0.15063 |
| Canada         |           7

In [40]:
#6. Top 5 Violations with Highest Arrest Rates

mycursor.execute("""WITH rate_table as
                    (SELECT violation,
                           (COUNT(CASE WHEN is_arrested=1 THEN 1 END) / COUNT(*)) * 100.0 AS arrest_rate
                    FROM police_log
                    GROUP BY violation)
                    SELECT violation, arrest_rate, RANK() OVER(ORDER BY arrest_rate DESC) AS rate_rank
                    FROM rate_table LIMIT 5
                    """)
all_rows=mycursor.fetchall()
print(tabulate(all_rows,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+-------------+---------------+-------------+
| violation   |   arrest_rate |   rate_rank |
|-------------+---------------+-------------|
| DUI         |       50.5545 |           1 |
| Seatbelt    |       50.5343 |           2 |
| Speeding    |       50.1749 |           3 |
| Signal      |       49.9466 |           4 |
| Other       |       49.3861 |           5 |
+-------------+---------------+-------------+


In [4]:
df=pd.read_sql_query("select * from police_log", connection)
df.head(5)

C:\Users\sidst\AppData\Local\Temp\ipykernel_18564\727775153.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query("select * from police_log", connection)


country_name driver_gender  driver_age_raw  driver_age driver_race  \
0       Canada             M              59          19       Asian   
1        India             M              35          58       Other   
2          USA             M              26          76       Black   
3       Canada             M              26          76       Black   
4       Canada             M              62          75       Other   

      violation_raw violation  search_conducted     search_type stop_outcome  \
0     Drunk Driving  Speeding                 1  Vehicle Search       Ticket   
1             Other     Other                 0  Vehicle Search       Arrest   
2  Signal Violation  Speeding                 0           Frisk       Ticket   
3          Speeding       DUI                 1           Frisk      Warning   
4          Speeding     Other                 0  Vehicle Search       Arrest   

   is_arrested  drugs_related_stop vehicle_number       stop_datetime  \
0            1                   1     UP76DY3473 2020-01-01 00:00:00   
1            1                   1     RJ83PZ4441 2020-01-01 00:01:00   
2            1                   1     RJ32OM7264 2020-01-01 00:02:00   
3            0                   1     RJ76TI3807 2020-01-01 00:03:00   
4            1                   0     WB63BB8305 2020-01-01 00:04:00   

   stop_duration_min  
0                 14  
1                 14  
2                 14  
3                 15  
4                 14